In [1]:
from __future__ import print_function
import pyrealsense2.pyrealsense2 as rs
import numpy as np
import cv2
import time
import os
import datetime
import skimage.filters
import pandas as pd
import torch
from transformers import BeitFeatureExtractor
from torchvision.transforms import Resize, Compose, Normalize, ToTensor
import glob
import pytorch_lightning as pl
# import mercury
# import RPi.GPIO as GPIO
# import multiprocessing as mp
# GPIO.setwarnings(False)
# GPIO.setmode(GPIO.BCM)
# GPIO.setup(13, GPIO.IN)
# from myFunctions import *

c:\Users\zacha\anaconda3\envs\ML_env_pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def image_preprocessing(depth_img, background):
    depth_img = background - depth_img #[:, int((848-640)/2):int(-(848-640)/2)]

    otsu_thresh = skimage.filters.threshold_otsu(depth_img)
    img_post = np.where(depth_img<otsu_thresh, depth_img, 0)

    img_post = np.where(img_post>600, depth_img, 0)

    img_mask = np.where(img_post != 0, 1, 0)
    img_seg = skimage.measure.label(img_mask, background = 0, connectivity=2)
    #assert( img_seg.max() != 0 )
    if img_seg.max() != 0:
        mask = img_seg == np.argmax(np.bincount(img_seg.flat)[1:])+1
    else:
        mask = np.zeros(np.shape(depth_img))

    depth_img = np.where(mask != 0, depth_img, 0)
    depth_img = (depth_img/np.max(depth_img)*255).astype('uint8')

    return depth_img, mask

    import pytorch_lightning as pl
from transformers import BeitForImageClassification, AdamW
import torch.nn as nn

class ViTLightningModule(pl.LightningModule):
    def __init__(self, num_labels=10):
        super(ViTLightningModule, self).__init__()
        self.vit = BeitForImageClassification.from_pretrained("microsoft/beit-base-patch16-224",
                                                              num_labels=5,
                                                            #   id2label=id2label,
                                                            #   label2id=label2id,
                                                              ignore_mismatched_sizes=True)

    def forward(self, pixel_values):
        outputs = self.vit(pixel_values=pixel_values)
        return outputs.logits
        
    # def common_step(self, batch, batch_idx):
    #     pixel_values = batch['pixel_values']
    #     labels = batch['labels']
    #     logits = self(pixel_values)

    #     #criterion = nn.CrossEntropyLoss()
    #     #criterion = nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor(class_weights))
    #     loss = criterion(logits, labels)
    #     predictions = logits.argmax(-1)
    #     correct = (predictions == labels).sum().item()
    #     accuracy = correct/pixel_values.shape[0]

    #     return loss, accuracy
      
    # def training_step(self, batch, batch_idx):
    #     loss, accuracy = self.common_step(batch, batch_idx)     
    #     # logs metrics for each training_step,
    #     # and the average across the epoch
    #     self.log("training_loss", loss)
    #     self.log("training_accuracy", accuracy)

    #     return loss
    
    # def validation_step(self, batch, batch_idx):
    #     loss, accuracy = self.common_step(batch, batch_idx)     
    #     self.log("validation_loss", loss, on_epoch=True)
    #     self.log("validation_accuracy", accuracy, on_epoch=True)

    #     return loss

    # def test_step(self, batch, batch_idx):
    #     loss, accuracy = self.common_step(batch, batch_idx)
    #     self.log("test_loss", loss, on_epoch=True)
    #     self.log("test_accuracy", accuracy, on_epoch=True)

    #     return loss, accuracy

    def configure_optimizers(self):
        # We could make the optimizer more fancy by adding a scheduler and specifying which parameters do
        # not require weight_decay but just using AdamW out-of-the-box works fine
        return AdamW(self.parameters(), lr=1e-5)

    # def train_dataloader(self):
    #     return train_dataloader

    # def val_dataloader(self):
    #     return val_dataloader
    
    # def test_dataloader(self):
    #     return test_dataloader
    
    # def predict_dataloader(self):
    #     return test_dataloader

In [3]:
fps = 30
width = 848
length = 480
total_pixels = width*length

transformer_name = "microsoft/beit-base-patch16-224"
model = ViTLightningModule.load_from_checkpoint(checkpoint_path="VIT_models\BEiT_42_keep_test.ckpt")
model.to('cpu')
model.eval()
feature_extractor = BeitFeatureExtractor.from_pretrained(transformer_name)
normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)

filenames = glob.glob('February_Data\\*.bag')
print(filenames)
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()
config.enable_device_from_file(filenames[4]) #4 is a good example
# align_to = rs.stream.color
# align = rs.align(align_to)
#config.enable_stream(rs.stream.depth, width, length, rs.format.z16, fps)
#config.enable_stream(rs.stream.color, width, length, rs.format.bgr8, fps)

pipe_profile = pipeline.start(config)

#decimation = rs.decimation_filter()
#decimation.set_option(rs.option.filter_magnitude, 2)

spatial = rs.spatial_filter()
spatial.set_option(rs.option.filter_magnitude, 5)
spatial.set_option(rs.option.filter_smooth_alpha, 1)
spatial.set_option(rs.option.filter_smooth_delta, 50)
spatial.set_option(rs.option.holes_fill, 2)

thresh = rs.threshold_filter(min_dist = 0.1, max_dist = 3.0)

# depth_sensor = pipe_profile.get_device().first_depth_sensor()

# depth_sensor.set_option(rs.option.enable_auto_exposure, True)

#getting the background when the camera is first turned on
frames = pipeline.wait_for_frames()
#aligned_frames = align.process(frames)
depth_frame = frames.get_depth_frame()
color_frame = frames.get_color_frame()
thresh_frame = thresh.process(depth_frame)
filtered_depth = spatial.process(thresh_frame)
background_image = np.asanyarray(filtered_depth.get_data())
print(np.shape(background_image))




while True:

    #getting frames from the camera
    frames = pipeline.wait_for_frames()
    #aligned_frames = align.process(frames)
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()
    thresh_frame = thresh.process(depth_frame)
    filtered_depth = spatial.process(thresh_frame)
    depth_image = np.asanyarray(filtered_depth.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    #import imageio
    #background_image = imageio.imread('masks\\feb_background.tif')
    cropped_depth, mask = image_preprocessing(depth_image, background_image)
    upper_pixels = np.sum(np.where(cropped_depth[:100, :] != 0, 1, 0))
    lower_pixels = np.sum(np.where(cropped_depth[length-20:length, :] != 0, 1, 0))
    total_nonzero_pixels = np.sum(np.where(cropped_depth != 0, 1, 0))
    
    
    display_img = np.zeros(np.shape(cropped_depth))
    display_img = np.where(cropped_depth !=0, cropped_depth, 0)
    # cv2.line(display_img,(0,100),(848,100),(255,0,0),1)
    # cv2.line(display_img,(0, length-20),(848, length-20),(255,0,0),1)
    
    # cv2.putText(display_img, 'Lower Pixel values: '+str(lower_pixels),(0,400), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255),2,cv2.LINE_AA)
    # cv2.putText(display_img, 'Upper Pixel values: '+str(upper_pixels),(0,150), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255),2,cv2.LINE_AA)

    # cv2.imshow('cropped depth', display_img)
    # cv2.waitKey(1)
    # print(lower_pixels, upper_pixels, total_nonzero_pixels)

    cropped_depth = (cropped_depth/np.max(cropped_depth)*255).astype('uint8')

    cropped_color = color_image
    cropped_color[:, :, 0] = np.where(mask==True, color_image[:, :, 0], 0)
    cropped_color[:, :, 1] = np.where(mask==True, color_image[:, :, 1], 0)
    cropped_color[:, :, 2] = np.where(mask==True, color_image[:, :, 2], 0)

    img_DGE = np.zeros(np.shape(color_image))
    img_DGE[:, :, 0] = cropped_depth
    img_DGE[:, :, 1] = skimage.color.rgb2gray(cropped_color)
    mask_dilation = skimage.morphology.binary_dilation(mask, skimage.morphology.ball(4, dtype=bool)[::, ::, 0], out=None)
    img_DGE[:, :, 2] = skimage.feature.canny(cropped_depth, sigma=1.0).astype('uint8')*255*mask_dilation
    #img_DGE= skimage.transform.rescale(img_DGE, (1920, 1080), anti_aliasing=True, channel_axis=2)
    # print(np.shape(img_DGE))
    transform = Compose([ToTensor(), Resize((224, 224)), normalize])
    img_DGE_norm = transform(img_DGE)

    outputs=model(img_DGE_norm[None, :].to('cpu').type('torch.FloatTensor'))
    prediction=np.argmax(outputs[0].cpu().detach().numpy(), axis=-1)
    #print(prediction)

    #cv2.putText(img_DGE, 'BCS: '+ str(prediction[0]),(0,400), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 0, 255),2,cv2.LINE_AA)
    cv2.putText(img_DGE,'Thi'+'c'*(prediction+2),(0,400), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 0, 255),2,cv2.LINE_AA)
    cv2.imshow('cropped color', cv2.resize(img_DGE, (1920,1080)))
    cv2.waitKey(1)


    # if (lower_pixels < 400) and (upper_pixels > 15000) and (total_nonzero_pixels > 14000):

    #     #cropped_depth = (cropped_depth/np.max(cropped_depth)*255).astype('uint8')

    #     cropped_color = color_image
    #     cropped_color[:, :, 0] = np.where(mask==True, color_image[:, :, 0], 0)
    #     cropped_color[:, :, 1] = np.where(mask==True, color_image[:, :, 1], 0)
    #     cropped_color[:, :, 2] = np.where(mask==True, color_image[:, :, 2], 0)

    #     #Now convert depth/color images to DGE
    #     img_DGE = np.zeros(np.shape(color_image))
    #     img_DGE[:, :, 0] = cropped_depth
    #     img_DGE[:, :, 1] = skimage.color.rgb2gray(cropped_color)
    #     mask_dilation = skimage.morphology.binary_dilation(mask, skimage.morphology.ball(4, dtype=bool)[::, ::, 0], out=None)
    #     img_DGE[:, :, 2] = skimage.feature.canny(cropped_depth, sigma=0.5).astype('uint8')*255*mask_dilation
    #     # print(np.shape(img_DGE))
    #     transform = Compose([ToTensor(), Resize((224, 224)), normalize])
    #     img_DGE_norm = transform(img_DGE)

    #     outputs=model(img_DGE_norm[None, :].to('cpu').type('torch.FloatTensor'))
    #     prediction=np.argmax(outputs[0].cpu().detach().numpy(), axis=-1)
    #     print(int(prediction))

    #     cv2.putText(img_DGE, 'Thiccness Index: '+'thi'+'c'*int(prediction),(0,400), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 0, 255),2,cv2.LINE_AA)
    #     cv2.imshow('cropped color', img_DGE)
    #     cv2.waitKey(1)
    

    #cv2.destroyAllWindows()

c:\Users\zacha\anaconda3\envs\ML_env_pytorch\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of BeitForImageClassification were not initialized from the model checkpoint at microsoft/beit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['February_Data\\Cow_10_0014.bag', 'February_Data\\Cow_11_0075.bag', 'February_Data\\Cow_12_6006.bag', 'February_Data\\Cow_13_8001.bag', 'February_Data\\Cow_14_1044.bag', 'February_Data\\Cow_1_8025.bag', 'February_Data\\Cow_2_6016.bag', 'February_Data\\Cow_3_6018.bag', 'February_Data\\Cow_4_8036.bag', 'February_Data\\Cow_5_8041.bag', 'February_Data\\Cow_6_5029.bag', 'February_Data\\Cow_7_6022.bag', 'February_Data\\Cow_8_2023.bag', 'February_Data\\Cow_9_NOTG.bag']
(480, 640)
